In [83]:
from bs4 import BeautifulSoup
import requests
import sqlite3
import time
import random

In [4]:
response = requests.get("https://myanimelist.net/users.php")
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
recentOnlineUsers = soup.find_all(class_='picSurround')

In [6]:
recentSlugs = [x.find('a').get('href') for x in recentOnlineUsers]

## Plan:

- For each user, find all their anime (status=7), rating for each. 
- Keep track of users, and user's watched anime in database
- Also keep track of unique animes, average score, popularity, etc.

## TODO:

think of how to design database

## Data to get from each user:

- Anime stats
    - "Days"
    - Mean score
    - Num watching
    - Num completed
    - Plan to watch
    - List watching
    - List completed
    - List Plan to watch


In [6]:
for profile in recentSlugs:
    profileResp = requests.get("https://myanimelist.net" + profile)
    profileSoup = BeautifulSoup(profileResp.text, 'html.parser')
    
    # Completed, etc append
    # "?status=n" where n = 1,2,3,4,5,6
    
    # class for stats looks like clearfix mb12 inside div with class stats anime

# Exploring

In [7]:
profile = recentSlugs[2]
profileResp = requests.get("https://myanimelist.net" + profile)
profileSoup = BeautifulSoup(profileResp.text, 'html.parser')

statsAnime = profileSoup.find(class_ = "stats anime")
clearfixmb12 = statsAnime.find_all(class_="clearfix mb12")
len(clearfixmb12)
clearfixmb12

# watching, completed, on hold, dropped, plan to watch
# hopefully, in that order.

clearfixmb12[0].find('a').text # 'watching'
clearfixmb12[0].find('span').text # '6'

statScore = statsAnime.find(class_="stat-score")
statScore = [x for x in list(statScore.children) if x != '\n'] # get two subdivs, without newlines
statScore[0].find('span').text # 'Days: '
statScore[0].text.split(': ') # ['Days', '213.4']
statScore[1].text.split(': ') # ['Mena Score', '6.92']


# list of watching:
clearfixmb12[0].find('a').get('href')


'https://myanimelist.net/animelist/isanybodyhome?status=1'

Getting a list of watching, completed, etc anime is a bit trickier because the site is an infinite scrolling type. It loads at first 300 results, but you have to scroll to get more. 

https://myanimelist.net/animelist/Catalano?status=7

This will be a case study because they've watched a lot of anime.

The simple, but slow approach is to use Selenium to drive an actual browser to scroll and load the data. When it's all done, use BeautifulSoup to read everything. 

However, I don't want to use Selenium in this project, because I think it's a bit bulky. Since I am only scraping from one site, I figure I can do a bit of hardcoding. Theoretically, the data has to come from somewhere. 

https://blog.michaelyin.info/how-crawl-infinite-scrolling-pages-using-python/

Is a great post on finding the request URL the site makes to the backend to retrieve the data.

It looks like the response URL is https://myanimelist.net/animelist/Catalano/load.json?offset=300&status=7 for after the first 300. 

Going to use Python to dig deeper on how the JSON is structured

In [8]:
import json

offset0 = requests.get('https://myanimelist.net/animelist/Catalano/load.json?offset=0&status=7')
json.loads(offset0.text)[0]

offset300 = requests.get('https://myanimelist.net/animelist/Catalano/load.json?offset=300&status=7')
json.loads(offset300.text)[0]

{'status': 2,
 'score': 7,
 'tags': '',
 'is_rewatching': 0,
 'num_watched_episodes': 1,
 'anime_title': 'Gatchaman Crowds Insight: Inbound',
 'anime_num_episodes': 1,
 'anime_airing_status': 2,
 'anime_id': 30925,
 'anime_studios': None,
 'anime_licensors': None,
 'anime_season': None,
 'has_episode_video': False,
 'has_promotion_video': False,
 'has_video': False,
 'video_url': '/anime/30925/Gatchaman_Crowds_Insight__Inbound/video',
 'anime_url': '/anime/30925/Gatchaman_Crowds_Insight__Inbound',
 'anime_image_path': 'https://myanimelist.cdn-dena.com/r/96x136/images/anime/2/74427.jpg?s=31f643601049374df4ee968b99f0c2d9',
 'is_added_to_list': False,
 'anime_media_type_string': 'ONA',
 'anime_mpaa_rating_string': 'PG-13',
 'start_date_string': None,
 'finish_date_string': None,
 'anime_start_date_string': '20-06-15',
 'anime_end_date_string': '20-06-15',
 'days_string': None,
 'storage_string': '',
 'priority_string': 'Low'}

`status: 2` probably refers to it being marked "completed". `status: 1` probably refers to it being marked "currently watching"

It looks like if we set `offset=0` then we get the 1-300 (inclusive) anime watched. If we set `offset=300` then we get the 301-600 animes watched. What happens if the offset is more than they've watched?

In [17]:
offset3000 = requests.get('https://myanimelist.net/animelist/Catalano/load.json?offset=3000&status=7')
json.loads(offset3000.text)

[]

In [10]:
animeSoup = BeautifulSoup(requests.get('https://myanimelist.net/anime/31646/').text, 'html.parser')

# what anime information do we want?
# Score
# Genres

# getting genres

testlist = [x for x in animeSoup.find_all(class_="dark_text") if 'Genres' in x.text]
pardiv = testlist[0].parent
listOfGenres = [x.text for x in pardiv.find_all('a')]

# getting Score
score = animeSoup.find(itemprop="ratingValue").text


In [11]:
scorediv = [x.parent for x in animeSoup.find_all(class_="dark_text") if 'Ranked' in x.text][0]
scorediv

<div class="spaceit po-r js-statistics-info di-ib" data-id="info2">
<span class="dark_text">Ranked:</span>
  #143<sup>2</sup>
<div class="statistics-info info2">
<small><sup>2</sup>
    based on the <a href="/topanime.php">top anime</a> page. Please note that 'Not yet aired' and 'R18+' titles are excluded.
    </small>
</div>
</div>

We get an empty list when we try to ask for more than they've watched. I think a simple try/catch block on a list index out of range should handle the error well enough. 

I think I have a good way to get all the data I need, all that's left is to store it. Since the plan is to run a script periodically, I want my data to persist, so I'll try using a database. 

I'm also new to database, so this is also a first attempt at databases.

What I want to store is many users and many anime that have a many to many relationship, in that users watch many anime and an anime is watched by many users. It took some time to manage to formulate this idea with this language, but once I did, I found that I need to use a junction table (in a relational database world) to achieve this sort of relationship. The lack of "tuples" or "lists" in SQL was really messing with my brain.

In [45]:
profile = recentSlugs[2]

#def writeToDB(profileurl):
# Days
# Mean Score
# Watch list
# Completed list
# Plan to watch list

profileResp = requests.get("https://myanimelist.net" + profile)
profileSoup = BeautifulSoup(profileResp.text, 'html.parser')

username = profile.split('/')[-1]

# if username already here, just stop TODO

# Get days and mean score
statsAnime = profileSoup.find(class_ = "stats anime")
statScore = statsAnime.find(class_="stat-score")
statScore = [x for x in list(statScore.children) if x != '\n'] # get two subdivs, without newlines
statScore[0].find('span').text # 'Days: '

numdays = int(float(statScore[0].text.split(': ')[1])) # ['Days', '213.4']
meanscore = int(float(statScore[1].text.split(': ')[1])) # ['Mea Score', '6.92']

runninglist = []

# add it to SQL table

# First, try to add the user. If this user is already in the database
# (cursor will raise an integrity error), we can skip it. 

try:
    conn = sqlite3.connect('testdb.db')
    c = conn.cursor()
    with conn:
        c.execute('insert into users values ( "{un}", {nd}, {ms} );'\
                 .format(un=username, nd=numdays, ms=meanscore))
        
        # At this point, the user should have been added.
        # Now, we gather anime data.
        
        animelistURL = 'https://myanimelist.net/animelist/' + username
        currPage = 0
        
        # there is a chance that the server acts up. Hopefully, it doesn't. If it does, catch the exception.
        try:
            while True:
                listRequest = requests.get(animelistURL + '/load.json?offset=' + str(currPage*300) + '&status=7')
                animelist = json.loads(listRequest.text)
                if animelist == []:
                    break
                runninglist += animelist
                currPage += 1
                time.sleep(1)
        except ValueError:
            print("error: below is request text:")
            print(listRequest.text)
            print(listRequest)
            
        # Let's take a look at each anime, and commit to the database.
        
        for anime in animelist:
            # row structure for animes table is 
            # ID (int) | animename (TEXT) | score (REAL)
            
            animeID = anime['anime_id']
            animename = anime['anime_name']
            statusID = anime['status']
            userscore = anime['score']
            
            # need to get average score. 
            animeURL = 'http://myanimelist.net/anime/' + animeID
            try:
                animeRequest = requests.get(animeURL, 'html.parser')
                animeRequest.raise_for_status() # raise exception if something is up, else gucci.
                animeSoup = BeautifulSoup(animeRequest.text, 'html.parser')
                animeScore = animeSoup.find(itemprop="ratingValue").text
                
                # get Genres
                testlist = [x for x in animeSoup.find_all(class_="dark_text") if 'Genres' in x.text]
                pardiv = testlist[0].parent
                listOfGenres = [x.text for x in pardiv.find_all('a')]
    
            except requests.exceptions.HTTPError as err:
                print("An HTTP error has occured:")
                print(err)

                

except sqlite3.IntegrityError:
    print("UNIQUE constraint failed. " + username + " already in database. Continuing...")
    # need to put continue here, if no commands in except statement.

print("outside of try/catch")

print("number of animes for user " + username + " is: " + str(len(runninglist)))
        


outside of try/catch
number of animes for user isanybodyhome is: 233


In [42]:
conn = sqlite3.connect('testdb.db')
c = conn.cursor()
c.execute('select * from users')
print(c.fetchall())
c.close()

[('Alex82829290', 48.0, 8.0), ('Jrdotan', 112.0, 6.0)]


In [ ]:
# get big animelist


try:
    while True:
        listRequest = requests.get(animelistURL + 'load.json?offset=' + currPage + '&status=7')
        animelist = json.loads(listRequest.text)
        
        for anime in animelist:
            # Add to junction table and anime table
        

In [22]:
import time # prevent server from denying requests
username = 'Catalano'
animelistURL = 'https://myanimelist.net/animelist/' + username
currPage = 0
runninglist = []

try:
    while True:
        listRequest = requests.get(animelistURL + '/load.json?offset=' + str(currPage*300) + '&status=7')
        animelist = json.loads(listRequest.text)
        if animelist == []:
            break
        runninglist += [x['anime_title'] for x in animelist]
        currPage += 1
        time.sleep(1)
except ValueError:
    print("error: below is request text:")
    print(listRequest.text)
    print(listRequest)

print("success")


success


In [14]:
len(runninglist)

1104

In [92]:
import time # prevent server from denying requests
username = 'Catalano'
animelistURL = 'https://myanimelist.net/animelist/' + username
currPage = 0
runninglist = ()

In [105]:
listRequest = requests.get(animelistURL + '/load.json?offset=' + str(currPage*300) + '&status=7')
animelist = json.loads(listRequest.text)
runninglist = runninglist + tuple(x['anime_title'] for x in animelist)
currPage += 1

In [106]:
len(runninglist)

1104

In [110]:
listRequest.textthu

'[]'

In [13]:
conn = sqlite3.connect('testdb.db')
c = conn.cursor()
c.execute('insert into users values ( "{un}", {nd}, {ms} );'\
         .format(un=username, nd=numdays, ms=meanscore))
conn.commit()
conn.close()

In [14]:
conn = sqlite3.connect('testdb.db')
c = conn.cursor()

c.execute("select * from users;")
print(c.fetchall())
conn.commit()
conn.close()

[('MangaReader225', 56, 6)]


In [46]:
runninglist[120]

{'status': 2,
 'score': 9,
 'tags': '',
 'is_rewatching': 0,
 'num_watched_episodes': 3,
 'anime_title': 'Mahoutsukai no Yome: Hoshi Matsu Hito',
 'anime_num_episodes': 3,
 'anime_airing_status': 2,
 'anime_id': 32902,
 'anime_studios': None,
 'anime_licensors': None,
 'anime_season': None,
 'has_episode_video': True,
 'has_promotion_video': True,
 'has_video': True,
 'video_url': '/anime/32902/Mahoutsukai_no_Yome__Hoshi_Matsu_Hito/video',
 'anime_url': '/anime/32902/Mahoutsukai_no_Yome__Hoshi_Matsu_Hito',
 'anime_image_path': 'https://myanimelist.cdn-dena.com/r/96x136/images/anime/4/80587.jpg?s=2b7be10af2f96f8837763dcc3d738785',
 'is_added_to_list': False,
 'anime_media_type_string': 'OVA',
 'anime_mpaa_rating_string': 'PG-13',
 'start_date_string': None,
 'finish_date_string': None,
 'anime_start_date_string': '09-10-16',
 'anime_end_date_string': '09-09-17',
 'days_string': None,
 'storage_string': '',
 'priority_string': 'Low'}

In [114]:
animeurl = 'http://myanimelist.net/anime/' + '123'
animerequest = requests.get(animeurl, headers={'User-Agent': ua.random})
animeSoup = BeautifulSoup(animerequest.text, 'html.parser')
animerequest.raise_for_status()

In [64]:
animelist = runninglist

In [126]:
# this cell gets data for each anime in animelist. 
# TODO: make an async version.

numErrors = 0
errors = []
curranim_num = 1
from fake_useragent import UserAgent
ua = UserAgent()
for anime in animelist:
    # row structure for animes table is 
    # ID (int) | animename (TEXT) | score (REAL)

    animeID = anime['anime_id']
    animename = anime['anime_title']
    statusID = anime['status']
    userscore = anime['score']
    
    animeScore = -1

    # need to get average score. 
    animeURL = 'http://myanimelist.net/anime/' + str(animeID)
    try:
        animeRequest = requests.get(animeURL, 'html.parser', headers={'User-Agent': ua.random})
        animeRequest.raise_for_status() # raise exception if something is up, else gucci.
        animeSoup = BeautifulSoup(animeRequest.text, 'html.parser')
        
        try:
            animeScore = animeSoup.find(itemprop="ratingValue").text

            # get Genres
            testlist = [x for x in animeSoup.find_all(class_="dark_text") if 'Genres' in x.text]
            pardiv = testlist[0].parent
            listOfGenres = [x.text for x in pardiv.find_all('a')]
        except AttributeError as ae:
            print(ae)
            print("id when error occured: " + str(animeID))

    except requests.exceptions.HTTPError as err:
        print("An HTTP error has occured:")
        print(err)
        errors.append(animeRequest)
        numErrors += 1
        
    # print("Let's print all the data for this run.")
    # print("Username: " + username)
    # print("animeID: " + str(animeID))
    # print("animeName: " + animename)
    # print("status: " + str(statusID))
    # print("User rated this anime: " + str(userscore))
    # print("This anime's average score: " + str(animeScore))
    # print("This anime is tagged with the following genres: " + str(listOfGenres))
    
    print(curranim_num)
    curranim_num += 1
    time.sleep(5 + random.randint(-1, 1))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
An HTTP error has occured:
429 Client Error: Too Many Requests for url: http://myanimelist.net/anime/20847?html.parser
179
180
181
An HTTP error has occured:
429 Client Error: Too Many Requests for url: http://myanimelist.net/anime/21843?html.parser
182
183
184
185
An HTTP error has occured:
429 Client Error: Too Many Requests for url: https://myanimelist.net/anime/35629?html.parser
186
An HTT

In [88]:
from fake_useragent import UserAgent
ua = UserAgent()

In [122]:
animeURL = 'http://myanimelist.net/anime/37722'
animeRequest = requests.get(animeURL, 'html.parser', headers={'User-Agent': ua.random})
animeRequest.raise_for_status() # raise exception if something is up, else gucci.
animeSoup = BeautifulSoup(animeRequest.text, 'html.parser')
animeScore = animeSoup.find(itemprop="aggregateRating")



In [167]:
# try to empirically find max request rate 

import time # prevent server from denying requests
username = 'Catalano'
animelistURL = 'https://myanimelist.net/animelist/' + username
currPage = 0
runninglist = []
try:
    while True:
        listRequest = requests.get(animelistURL + '/load.json?offset=' + str(currPage*300) + '&status=7')
        print(listRequest)
        animelist = json.loads(listRequest.text)
        if animelist == []:
            break
        runninglist += animelist
        currPage += 1
except:
    print("an exception has been raised")
    
print("success")

# now that we have running list, let's try to hit every page.

        


<Response [200]>
<Response [429]>
an exception has been raised
success


In [168]:
len(runninglist)

300

In [169]:

deniedAccess=False
consecHit = 0
start = time.clock()

for anim in runninglist:
    try:
        animReq = requests.get('http://myanimelist.net/anime/' + str(anim['anime_id']))
        animReq.raise_for_status()
        
        
        if deniedAccess == True:
            deniedAccess = False
            start = time.clock()

        consecHit += 1
        time.sleep(0.5)
        
    except requests.HTTPError as he:
        if deniedAccess == True:
            print("=============================")
            print("Attribute Error exception")
            print(he)
            print("Number of consecutive hits: " + str(consecHit))
            print("Time since last fail: " + str(time.clock() - start))
            consecHit = 0
        else: 
            deniedAccess = True

Attribute Error exception
429 Client Error: Too Many Requests for url: http://myanimelist.net/anime/37105
Number of consecutive hits: 0
Time since last fail: 0.016617999999994026
Attribute Error exception
429 Client Error: Too Many Requests for url: http://myanimelist.net/anime/37259
Number of consecutive hits: 0
Time since last fail: 0.02453599999998346
Attribute Error exception
429 Client Error: Too Many Requests for url: http://myanimelist.net/anime/35983
Number of consecutive hits: 0
Time since last fail: 0.030673000000007278
Attribute Error exception
429 Client Error: Too Many Requests for url: http://myanimelist.net/anime/34612
Number of consecutive hits: 0
Time since last fail: 0.03838700000000017
Attribute Error exception
429 Client Error: Too Many Requests for url: http://myanimelist.net/anime/34494
Number of consecutive hits: 0
Time since last fail: 0.045761999999996306
Attribute Error exception
429 Client Error: Too Many Requests for url: http://myanimelist.net/anime/35994
N

In [131]:
runninglist

['D-Frag!',
 'Grand Blue',
 'Hanebado!',
 'Harukana Receive',
 'Saiki Kusuo no Ψ-nan (TV) 2',
 'Sakura Quest',
 'Satsuriku no Tenshi',
 'Sekai Seifuku: Bouryaku no Zvezda',
 'Shingeki no Kyojin Season 3',
 'Shirobako',
 'Shuumatsu no Izetta',
 'Steins;Gate 0',
 'Sumomomo Momomo: Chijou Saikyou no Yome',
 'Uchouten Kazoku 2',
 'Urusei Yatsura',
 'Yawara!',
 '"Bungaku Shoujo" Kyou no Oyatsu: Hatsukoi',
 '"Bungaku Shoujo" Memoire',
 '"Bungaku Shoujo" Movie',
 '009-1',
 '009-1: R&B',
 '3-gatsu no Lion',
 '3-gatsu no Lion 2nd Season',
 '91 Days',
 'A Kite',
 'Abenobashi Mahou☆Shoutengai',
 'Afro Samurai',
 'Afro Samurai Movie',
 'Afro Samurai: Resurrection',
 'Aho Girl',
 'Ajin Part 1: Shoudou',
 'Akira',
 'Aldnoah.Zero',
 'Alice to Zouroku',
 'Alien 9',
 'Amanchu!',
 'Amanchu! Advance',
 'Amanchu!: Yakusoku no Natsu to Atarashii Omoide no Koto',
 'Angel Beats!',
 'Angel Beats! Specials',
 'Angel Densetsu',
 'Angel Heart',
 'Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai.',
 'Ano H